In [1]:
import numpy as np
import csv
import pandas
import random
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense , Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import collections
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
filename1='C:/Users/RAHUL/Downloads/motivation - Sheet1.csv'
#filename2='C:/Users/RAHUL/Downloads/calm.csv'
filename3='C:/Users/RAHUL/Downloads/happy - Sheet1.csv'
filename='C:/Users/RAHUL/Downloads/compiled.csv'


In [10]:
def read_csv_shuffle(filename):
    dataframe = pandas.read_csv(filename, header=None)
    ds = dataframe.sample(frac=1)
    dataset = ds.values
    X = dataset[:,0]
    Y = dataset[:,1]
    return X,Y
X,Y=read_csv_shuffle(filename)

In [14]:
def generate_y_labels(Y):
    y_train=[]
    for var in Y:
        if(var=="Happy"):
            y_train.append(1)
        else:
            y_train.append(0)
    return y_train
y_train=generate_y_labels(Y)

In [19]:
def generate_all_sentences(X):
    sentences=[]
    for row in X:
        #print(row)
        sentences.append(tf.compat.as_str(row).split())
    return sentences
sentences=generate_all_sentences(X)
#print(sentences[6]) 
#print(y_train[6])

In [20]:
def get_word_list(sentences):
    word_list=[]
    for sentence in sentences:
        for word in sentence:
            word_list.append(word)
    return word_list
#split=sentence.split()
#print(split)
word_list=get_word_list(sentences)
#print(word_list)

In [21]:
#I need to convert sentences to dictionary form
#for that I need a dict
def file_to_word_ids(word_list,sentences,word_to_id):
    new_sentences=[]
    for sentence in sentences:
        new_sentences.append([word_to_id[word] for word in sentence if word in word_to_id])
    #data = word_list
    return new_sentences

def get_dict(word_list,sentences):
    counter = collections.Counter(word_list)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    vocabulary = len(word_to_id)
    reverse_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))
    train_data = file_to_word_ids(word_list,sentences, word_to_id)
    return word_to_id,reverse_dictionary,vocabulary, train_data
dictionary,reverse_dictionary,vocabulary, X_train=get_dict(word_list,sentences)
print(len(X_train))
print(vocabulary)

230
782


In [22]:
max_review_length = 18
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)

In [23]:
X_test=X_train[192:224]
y_test=y_train[192:224]

In [24]:
X_train=X_train[:192]
#Y_train=Y_train[:160]
y_train=y_train[:192]

In [25]:
data_path='C:/Users/RAHUL/projects/Many_to_one'

In [26]:
# create the model
def build_model(hidden_size=32,use_dropout=True):
    #embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(vocabulary, hidden_size, input_length=max_review_length))
    #model.add(LSTM(hidden_size,return_sequences=True))
    model.add(LSTM(hidden_size))
    if use_dropout:
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model
def train_model(model,reload_filename='NULL',save_best_only=True,batch_size=32,num_epochs=10):
    if(save_best_only):
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1,monitor='val_acc', save_best_only=True, mode='auto')
    else:
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1)
    #period option in checkpointer -> after how many epochs to save the model
    if(reload_filename!="NULL"):
            model = load_model(data_path + "/" + reload_filename)
    model.fit(X_train, y_train, validation_split=0.3, epochs=num_epochs, batch_size=batch_size,callbacks=[checkpointer])
    model.save(data_path + "final_model.hdf5")
hidden_size=32
rnn_model=build_model()
#add call back to save model after every epoch

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 32)            25024     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 33,377
Trainable params: 33,377
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
train_model(model=rnn_model,save_best_only=True)

Train on 134 samples, validate on 58 samples
Epoch 1/10
134/134 [==============================] - 1s 10ms/step - loss: 0.6922 - acc: 0.5746 - val_loss: 0.6940 - val_acc: 0.4828

Epoch 00001: val_acc improved from -inf to 0.48276, saving model to C:/Users/RAHUL/projects/Many_to_one/model-01-0.6922.hdf5
Epoch 2/10
134/134 [==============================] - 0s 585us/step - loss: 0.6897 - acc: 0.5672 - val_loss: 0.6942 - val_acc: 0.4310

Epoch 00002: val_acc did not improve from 0.48276
Epoch 3/10
134/134 [==============================] - 0s 622us/step - loss: 0.6854 - acc: 0.5970 - val_loss: 0.6936 - val_acc: 0.4310

Epoch 00003: val_acc did not improve from 0.48276
Epoch 4/10
134/134 [==============================] - 0s 719us/step - loss: 0.6821 - acc: 0.5896 - val_loss: 0.6930 - val_acc: 0.4483

Epoch 00004: val_acc did not improve from 0.48276
Epoch 5/10
134/134 [==============================] - 0s 629us/step - loss: 0.6761 - acc: 0.6642 - val_loss: 0.6919 - val_acc: 0.4655

Epoch 

In [29]:
def test_model_accuracy(filename):
    model = load_model(data_path + "/" + filename)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))
filename="binary_crossentropy/single 32 lstm/model-10-0.5622.hdf5"
test_model_accuracy(filename)

32/32 [==============================] - 0s 8ms/step
Accuracy: 46.88%
